In [ ]:
import itertools
import logging
import time
import numpy as np
import math
import scipy.special
import pandas as pd
import seaborn as sns
import string

from qiskit.circuit.library import EfficientSU2
from qiskit import Aer
# from qiskit.validation.base import BaseModel, BaseSchema, ObjSchema, bind_schema, Obj
from qiskit.aqua import QuantumInstance
# from qiskit.aqua.operators import Z2Symmetries, WeightedPauliOperator
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA, SLSQP, L_BFGS_B
from qiskit.chemistry.fermionic_operator import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pyscf import gto, scf, ao2mo, fci, cc, mp, hci
from pyscf.hci import SelectedCI 
from pyscf.fci import cistring
import numpy as np
import itertools
from collections import Counter
import operator
import matplotlib.pyplot as plt
import pyscf
import h5py

In [ ]:
def convert_inds_string_list_ints(inds_string):
    
    exclude = set(string.punctuation)
    inds = ''.join(ch for ch in str(inds_string) if ch not in exclude)
    inds_list = [int(ind) for ind in inds.split()]
    
    return inds_list

def convert_t2_exc_indices_to_occ_virt_ind(inds, nelec, norb):
    # converts indices of double block excitations alpha, beta
    # to indices as occ, occ, virt, virt
    
    conv_inds = np.zeros((4), dtype=int)
    nocc = int(nelec/2)
#     print("inds =", inds)
    conv_list = list()
    for n in range(4):
        if inds[n] < nocc:
            conv_inds[n] = int(inds[n])

        if nocc <= inds[n] < norb:
            conv_inds[n] = int(inds[n] - nocc)
        
        if norb <= inds[n] < norb + nocc:
            conv_inds[n] = int(inds[n] - norb)
        
        if norb + nocc <= inds[n] < 2*norb:
            conv_inds[n] = int(inds[n] - norb - nocc)
        conv_list.append(conv_inds[n]) 
    
#     print("converted inds", conv_list)
    
    return list(conv_list)

def convert_t1_exc_indices_to_occ_virt_ind(inds, nelec, norb):
    # converts indices of single block excitations alpha, beta
    # to indices as occ, virt
    
    conv_inds = np.zeros((2), dtype=int)
    nocc = int(nelec/2)
#     print("inds =", inds)
    conv_list = list()
    for n in range(2):
        if inds[n] < nocc:
            conv_inds[n] = int(inds[n])

        if nocc <= inds[n] < norb:
            conv_inds[n] = int(inds[n] - nocc)
        
        if norb <= inds[n] < norb + nocc:
            conv_inds[n] = int(inds[n] - norb)
        
        if norb + nocc <= inds[n] < 2*norb:
            conv_inds[n] = int(inds[n] - norb - nocc)
        conv_list.append(conv_inds[n]) 
    
#     print("converted inds", conv_list)
    
    return list(conv_list)

    
def generate_t2_indices(norb, nelec, occ=None, t2=None):
    """
    Based on number of spatial orbitals and electrons generates: 
    1) Double excitations indices in block format i->a, j->b
    2) Dictionary where keys are spin-orbitals occupation numbers (determinants), values are indices from 1)
    
    By default uses RHF configuration
    norb: number of spatial orbitals, half the number of spin orbitals
    nelec: total number of electrons
    occ: occupation numbers for spin orbitals in block format (all alpha, all beta)
    """
    all_amps_dict = dict()
    amp_dict = dict()
    if occ is None:
        occ = np.zeros(norb*2, dtype=int)
        for n in range(int(nelec/2)):
            occ[n] = 1
            occ[n + norb] = 1
    
    # Indices for excitations from occupied to unoccupied
    i_inds = [i_ind for i_ind, element in enumerate(occ[0:norb]) if element > 0]
    j_inds = [j_ind + norb for j_ind, element in enumerate(occ[norb:]) if element > 0]
    a_inds = [a_ind for a_ind, element in enumerate(occ[0:norb]) if element == 0]
    b_inds = [b_ind + norb for b_ind, element in enumerate(occ[norb:]) if element == 0]

    inds_amps_dict = dict()
    t2_amp_vs_det_dict = dict()
    for exc_from_i in i_inds:
        for exc_from_j in j_inds:
            for exc_to_a in a_inds:
                for exc_to_b in b_inds:
                    
                    if exc_to_a > exc_from_i and exc_to_b > exc_from_j: # exciting only up, no deexcitation
                        # Maybe we can allow de-excitation here?
                        # computing new occupation numbers
                        fin_occ = occ.copy()
                        fin_occ[exc_from_i] -= 1
                        fin_occ[exc_to_a] += 1
                        fin_occ[exc_from_j] -= 1
                        fin_occ[exc_to_b] += 1

                        # appending array with proper indices and dictionary with occ numbers and indices
                        ind = [exc_from_i, exc_to_a, exc_from_j, exc_to_b]
                        ind_str = str(ind) 
                        ind_str_flipped = str([exc_from_j, exc_to_b, exc_from_i, exc_to_a])
                        
                        conf_string = "".join([str(occ_n) for occ_n in fin_occ])
                        if conf_string in t2_amp_vs_det_dict:
                            print("ABORT!!!!!!!!!!!!!!!!!")
#                         print(conf_string[:norb], conf_string[norb:], ind)
#                         if conf_string in t2_amp_vs_det_dict:
#                             print("already exists (alpha beta)")
#                             t2_amp_vs_det_dict[conf_string].append(ind)
#                         else:
                        t2_amp_vs_det_dict[conf_string] = ind
                        
                        if t2 is not None:

                            conv_inds = convert_t2_exc_indices_to_occ_virt_ind([exc_from_i, exc_from_j,
                                                                        exc_to_a, exc_to_b], nelec, norb)
                            cur_amp = t2[conv_inds[0], conv_inds[1], conv_inds[2], conv_inds[3]]
                            
                            if ind_str_flipped not in inds_amps_dict.keys():
                                inds_amps_dict[ind_str] = cur_amp
                            
                            if conf_string in all_amps_dict:
                                all_amps_dict[ind_str].append(tuple([ind, cur_amp]))
                            else:
                                all_amps_dict[ind_str] = [tuple([ind, cur_amp])]
                                
                            amp_dict[conf_string] = cur_amp
#                         print(t2_amp_vs_det_dict[conf_string])
                    else:
                        continue
    
    # excitations from both alpha 
    
    pair_inds_i = []
    for i in i_inds:
        for j in i_inds:
            if i != j:
                pair_inds_i.append((i, j))
    
    pair_inds_a = []
    for a in a_inds:
        for b in a_inds:
            if a != b:
                pair_inds_a.append((a, b))
    
#     print("pair inds i", pair_inds_i)
#     print("pair inds a", pair_inds_a)
    for exc_from_i, exc_from_j in pair_inds_i:
        for exc_to_a, exc_to_b in pair_inds_a:
#             print("ijab", exc_from_i, exc_from_j, exc_to_a, exc_to_b)
            if exc_to_a > exc_from_i and exc_to_b > exc_from_j: # exciting only up, no deexcitation
                # Maybe we can allow de-excitation here?
                # computing new occupation numbers
                fin_occ = occ.copy()
                fin_occ[exc_from_i] -= 1
                fin_occ[exc_to_a] += 1
                fin_occ[exc_from_j] -= 1
                fin_occ[exc_to_b] += 1

                # appending array with proper indices and dictionary with occ numbers and indices
                ind = [exc_from_i, exc_to_a, exc_from_j, exc_to_b]
                ind_str = str(ind)
                ind_str_flipped = str([exc_from_j, exc_to_b, exc_from_i, exc_to_a])

#                 inds.append(ind)
                conf_string = "".join([str(occ_n) for occ_n in fin_occ])
#                 print("conf_string", conf_string, "ind", ind)
                if conf_string in t2_amp_vs_det_dict:
#                     print("\nalready exists (alpha alpha)")
#                     t2_amp_vs_det_dict[conf_string].append(ind)
                      pass
                else:
                    t2_amp_vs_det_dict[conf_string] = ind
                
                if t2 is not None:
#                     print(conf_string[:norb], conf_string[norb:])
                    #!!!!!! for now keeping largest amplitude
                    conv_inds = convert_t2_exc_indices_to_occ_virt_ind([exc_from_i, exc_from_j,
                                                                        exc_to_a, exc_to_b], nelec, norb)
                    
                    cur_amp = t2[conv_inds[0], conv_inds[1], conv_inds[2], conv_inds[3]]
                
                    if ind_str_flipped not in inds_amps_dict.keys():
                        inds_amps_dict[ind_str] = cur_amp
                    
                    if conf_string in all_amps_dict:
                        all_amps_dict[ind_str].append(tuple([ind, cur_amp]))
                    else:
                        all_amps_dict[ind_str] = [tuple([ind, cur_amp])]
                                
#                     print("new t2 alpha",  
#                         t2[conv_inds[0], conv_inds[1], conv_inds[2], conv_inds[3]])
                    if conf_string in amp_dict:
#                         print("old t2", amp_dict[conf_string])
                        # trying to keep smaller ones
                        if np.abs(amp_dict[conf_string]) > cur_amp:
#                             print("keeping old")
                            pass
                        else:
#                             print("keeping new")
                            amp_dict[conf_string] = cur_amp
                        
                    else:
#                         print("keeping new")
                        amp_dict[conf_string] = cur_amp

            else:
                continue
                
                
        # NOT SURE if this makes any sense, since alpha and beta are only conventions
#     excitations from both beta 
        
    pair_inds_j = []
    for i in j_inds:
        for j in j_inds:
            if i != j:
                pair_inds_j.append((i, j))
    
    pair_inds_b = []
    for a in b_inds:
        for b in b_inds:
            if a != b:
                pair_inds_b.append((a, b))

    for exc_from_i, exc_from_j in pair_inds_j:
        for exc_to_a, exc_to_b in pair_inds_b:
#             print("ijab", exc_from_i, exc_from_j, exc_to_a, exc_to_b)
            if exc_to_a > exc_from_i and exc_to_b > exc_from_j: # exciting only up, no deexcitation
                # Maybe we can allow de-excitation here?
                # computing new occupation numbers
                fin_occ = occ.copy()
                fin_occ[exc_from_i] -= 1
                fin_occ[exc_to_a] += 1
                fin_occ[exc_from_j] -= 1
                fin_occ[exc_to_b] += 1

                # appending array with proper indices and dictionary with occ numbers and indices
                ind = [exc_from_i, exc_to_a, exc_from_j, exc_to_b]
                ind_str = str(ind)
                ind_str_flipped = str([exc_from_j, exc_to_b, exc_from_i, exc_to_a])

#                 inds.append(ind)
                conf_string = "".join([str(occ_n) for occ_n in fin_occ])
                
                # !!!! The following is problematic, different amplitudes corresponding to the same 
                # determinant have different magnitudes!
                if conf_string in t2_amp_vs_det_dict:
#                     print("already exists (beta beta)")
#                     t2_amp_vs_det_dict[conf_string].append(ind)
                    pass
                else:
                    t2_amp_vs_det_dict[conf_string] = ind

                if t2 is not None:
                    #!!!!!! for now keeping largest amplitude
#                     print(conf_string[:norb], conf_string[norb:])
                    conv_inds = convert_t2_exc_indices_to_occ_virt_ind([exc_from_i, exc_from_j,
                                                                        exc_to_a, exc_to_b], nelec, norb)
#                     print("new t2 beta",  
#                         t2[conv_inds[0], conv_inds[1], conv_inds[2], conv_inds[3]])
                    cur_amp = t2[conv_inds[0], conv_inds[1], conv_inds[2], conv_inds[3]]

                    if ind_str_flipped not in inds_amps_dict.keys():
                        inds_amps_dict[ind_str] = cur_amp

                    if conf_string in all_amps_dict:
                        all_amps_dict[ind_str].append(tuple([ind, cur_amp]))
                    else:
                        all_amps_dict[ind_str] = [tuple([ind, cur_amp])]
                        
                    if conf_string in amp_dict:
#                         print("old t2", amp_dict[conf_string])
                        # trying to keep smaller ones
                        if np.abs(amp_dict[conf_string]) > cur_amp:
#                             print("keeping old")
                            pass
                        else:
#                             print("keeping new")
                            amp_dict[conf_string] = cur_amp
                        
                    else:
#                         print("keeping new")
                        amp_dict[conf_string] = cur_amp

            else:
                continue

    
    return inds_amps_dict, t2_amp_vs_det_dict, amp_dict, all_amps_dict

def generate_t1_indices(norb, nelec, occ=None, t1=None):
    """
    Generates: 
    1) Single excitations indices in block format i->a, j->b
    2) Dictionary where keys are spin-orbitals occupation numbers (determinants), values are indices from 1)
    
    By default uses RHF configuration
    norb: number of spatial orbitals, half the number of spin orbitals
    nelec: total number of electrons
    occ: occupation numbers for spin orbitals in block format (all alpha, all beta)
    """
    
    if occ is None:
        occ = np.zeros(norb*2, dtype=int)
        for n in range(int(nelec/2)):
            occ[n] = 1
            occ[n + norb] = 1
    inds_amps_dict = dict()
    amp_vs_det_dict = dict()
    amp_dict = dict()
    
    i_inds = [i_ind for i_ind, element in enumerate(occ[0:norb]) if element > 0]
    j_inds = [j_ind + norb for j_ind, element in enumerate(occ[norb:]) if element > 0]
    a_inds = [a_ind for a_ind, element in enumerate(occ[0:norb]) if element == 0]
    b_inds = [b_ind + norb for b_ind, element in enumerate(occ[norb:]) if element == 0]
    
    # single excitations for alpha orbitals
    for exc_from_i in i_inds:
        for exc_to_a in a_inds:
            ini_occ = occ.copy()
            
            if exc_to_a > exc_from_i:
                ini_occ = occ.copy()
                ini_occ[exc_from_i] -= 1
                ini_occ[exc_to_a] += 1
                string1 = "".join([str(a) for a in ini_occ])
                amp_vs_det_dict[string1] = [exc_from_i, exc_to_a]
                
                if t1 is not None:
                    inds_amps_dict[str([exc_from_i, exc_to_a])] = t1[exc_from_i, exc_to_a - int(nelec/2)]
                    amp_dict[string1] = t1[exc_from_i, exc_to_a - int(nelec/2)]
                    
    # single excitations for beta orbitals
    for exc_from_j in j_inds:
        for exc_to_b in b_inds:
            
            if exc_to_b > exc_from_j:
                ini_occ = occ.copy()
                ini_occ[exc_from_j] -= 1
                ini_occ[exc_to_b] += 1
                string2 = "".join([str(a) for a in ini_occ])
                amp_vs_det_dict[string2] = [exc_from_j, exc_to_b]
                
                if t1 is not None:
                    inds_amps_dict[str([exc_from_j, exc_to_b])] = t1[exc_from_j - norb, exc_to_b - int(nelec/2) - norb]
                    amp_dict[string2] = t1[exc_from_j - norb, exc_to_b - int(nelec/2) - norb]
            

    return inds_amps_dict, amp_vs_det_dict, amp_dict


In [ ]:
logging.basicConfig(level=logging.INFO)
K = 12
N = 4
total = 0
for n in range(1, N+1):
    total += scipy.special.comb(N, n)*scipy.special.comb(K-N, n)
    print("N configurations", scipy.special.comb(N, n)*scipy.special.comb(K-N, n))
print("total", total)
print(scipy.special.comb(K, N))

In [ ]:
def create_array_for_heatmap(det_vs_amp_dict):
    
    norb = len(list(det_vs_amp_dict.keys())[0])
    nelec = sum([int(a) for a in list(det_vs_amp_dict.keys())[0]])        
    n_occ = int(nelec/2)
    n_sp_orb = int(norb/2)
    
    alpha = []
    beta = []
    coeff = []
    
    for key in det_vs_amp_dict.keys():
        alpha.append(key[0:n_sp_orb])
        beta.append(key[n_sp_orb:])
        coeff.append(det_vs_amp_dict[key])

    return alpha, beta, coeff
        
def filter_out_n_order_exc_dets(det_vs_amp_dict, n):
    
    filtered_dets = []
    norb = len(list(det_vs_amp_dict.keys())[0])
    nelec = sum([int(a) for a in list(det_vs_amp_dict.keys())[0]])
    filtered_dets = dict()

#     print("norb", norb)
#     print("nelec", nelec)
    n_occ = int(nelec/2)
    n_sp_orb = int(norb/2)
    for key in det_vs_amp_dict.keys():
        exc_n_a = sum([int(a) for a in key[0:n_occ]])
        exc_n_b = sum([int(a) for a in key[n_sp_orb:n_sp_orb + n_occ]])
        
        exc = nelec - exc_n_a - exc_n_b
        if exc == n:
            filtered_dets[key] = det_vs_amp_dict[key].copy()
    
    return filtered_dets  

def compute_miltiplicity(det):
    # compute multiplicity for a determinant (block form)
    # accepts string with first half alpha, second half beta
    n_orb = int(len(det)/2)
    if n_orb % 2 != 0: print("Not a valid string")
    spin = 0
    for n in range(n_orb):
        spin += 1/2 * int(det[n])
        spin -= 1/2 * int(det[n + n_orb])
    mult = int(2 * spin + 1)
    return mult

def print_sorted_dict(dict_to_print):
    norb = int(len(list(dict_to_print.keys())[0]) / 2)
    sorted_dict = dict( sorted(dict_to_print.items(), key=lambda kv: np.max(np.abs(kv[1])), reverse=True))
    for key in sorted_dict.keys():
        print(key[0:norb], key[norb:], sorted_dict[key])

def filter_dets_from_dict(d, thresh=1e-08):
    filtered_dict = dict()
    for key in d.keys():
        if np.max(np.abs(d[key])) > thresh:

            filtered_dict[key] = d[key]
        # else:
        #     print(key, np.max(np.abs(d[key])))
    # print("filtered", filtered_dict)
    return filtered_dict  

def filter_out_excitations_from_det_dict(dict_to_filter, order_of_exc, nelec):
    filtered_dict = dict()
    nocc = int(nelec/2)
    for key in dict_to_filter.keys():
        n_el_occ = 0
        for x in key[0:nocc]:
            n_el_occ += int(x)
        # print("n_el_occ", n_el_occ, key)
        if nelec - n_el_occ == order_of_exc:
            filtered_dict[key] = dict_to_filter[key]
    return filtered_dict


    return det_vs_amp_ind_dict

def run_calcs(r, mol, basis, sci_select_cutoff, sci_ci_coeff_cutoff, det_thresh, discarded_amp_thresh, mp2_thresh):
   
    print("r=", r)
    basis = basis
    sci_select_cutoff = sci_select_cutoff
    sci_ci_coeff_cutoff = sci_ci_coeff_cutoff
    det_thresh = det_thresh # filters SCI selected dets (in FCI format there's a lot of zeros)
    discarded_amp_thresh = discarded_amp_thresh # for counting amplitudes filtered out by SCI
    mp2_thresh = mp2_thresh # throw out mp amps smaller than this 

    mol.verbose = 0

    # RHF
    mf = scf.RHF(mol)
#     mf = scf.UHF(mol)
    e_hf = mf.kernel()
    h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
    eri = ao2mo.kernel(mol, mf.mo_coeff)
    
    
    # Full CI
    print("\nFCI")
    cisolver = fci.FCI(mol)

    e_fci, ci = cisolver.kernel(h1, eri, h1.shape[1], mol.nelec, ecore=mol.energy_nuc())
#     e_fci, ci = cisolver.kernel(ecore=mol.energy_nuc())
    
    print("FCI energy", e_fci)
    norb = h1.shape[1]
    nelec = mol.nelectron
    print("nelec", nelec)
    print("norb", norb)
#     print("Num Strings: ", cistring.num_strings(norb,nelec/2))
    print("Spin contamination in FCI:\nS^2", cisolver.spin_square(ci, norb, nelec=(int(nelec/2), int(nelec/2))))
    
    spin_orb_strings = list()
    det_dict = dict()
    det_dict_spin_orb = dict()
    
    for alpha_addr in range(ci.shape[0]):
        for beta_addr in range(ci.shape[0]):
            # In pyscf the lower energy orbitals are to the RIGHT!!! so we have to reverse the strings

            a_str = bin(cistring.addrs2str(norb, int(nelec/2),[alpha_addr])[0])[2:].rjust(norb, '0')[::-1] # converting to strings with occ numbers
            b_str = bin(cistring.addrs2str(norb, int(nelec/2),[beta_addr])[0])[2:].rjust(norb, '0')[::-1]
            
            spin_orb_str = ''
            spin_orb_str = a_str + b_str
            
            # for spin orbitals
            det_dict_spin_orb[spin_orb_str] = [ci[alpha_addr, beta_addr]]
            
    print("Number of FCI determinants", len(det_dict_spin_orb.keys()))

    # Running CCSD
    print("\nCCSD")
    mycc = cc.CCSD(mf)
    result = mycc.kernel()
    t2 = result[2]
    t1 = result[1]
    ccsd_conv = result[0]

    t2_inds_ccsd_dict, det_vs_amp_ind, amp_dict, all_amps_ccsd = generate_t2_indices(norb, nelec, t2=t2)
    t1_inds_ccsd_dict, det_vs_amp_t1_ind, amp_dict_t1 = generate_t1_indices(norb, nelec, t1=t1)

    print("Number of CCSD t1 dets", len(det_vs_amp_t1_ind.keys()))
    print("Number of CCSD t2 dets", len(det_vs_amp_ind.keys()))
    
    
    # MP2
    print("\nMP2")
    pt = mp.MP2(mf).run()
    mp2 = pt.run()

    t2_inds_mp2_dict, mp2_dict, amp_dict_mp2, all_amps_mp2 = generate_t2_indices(norb, nelec, t2=mp2.t2)

    print("number of MP2 t2 dets", len(mp2_dict.keys()))

    t2_filtered_mp2 = mp2.t2.copy()
    mp2_amps_to_keep = []
    mp2_amp_inds_smaller_than_thresh = []
    for idx_str in t2_inds_mp2_dict.keys():
        
        idx = convert_inds_string_list_ints(idx_str)
#         print(idx)
        conv_idx = convert_t2_exc_indices_to_occ_virt_ind(idx, nelec, norb)
        if [idx[2], idx[3], idx[0], idx[1]] in mp2_amps_to_keep:
            print("FLIP")
            continue
            
        if np.abs(mp2.t2[conv_idx[0], conv_idx[2], conv_idx[1], conv_idx[3]]) < mp2_thresh:
            
#             print("MP2 t2 shape", np.shape(t2_filtered_mp2))
#             print("IDX", conv_idx)
            # this function spits out ijab order with occ, occ, virt, virt dimensions
            t2_filtered_mp2[conv_idx[0], conv_idx[2], conv_idx[1], conv_idx[3]] = 0
            mp2_amp_inds_smaller_than_thresh.append(idx)
        else:
            mp2_amps_to_keep.append(idx)
#     print("mp2 amps to discard", mp2_amp_inds_smaller_than_thresh)
    zero_mp2_indices = np.abs(mp2.t2) < mp2_thresh
    t2_filtered_mp2 = t2.copy()
    t2_filtered_mp2[zero_mp2_indices] = 0
    
#     #### DEBUG#######
#     # For LiH    
#     tmp1 = convert_t2_exc_indices_to_occ_virt_ind([0,1,2,5], 4, 6)
#     tmp2 = convert_t2_exc_indices_to_occ_virt_ind([0,1,5,2], 4, 6)
#     print(tmp1)
#     print(tmp2)
#     print(np.shape(t2))
#     print(t2[tmp1[0], tmp1[1], tmp1[2], tmp1[3]], t2[tmp2[0], tmp2[1], tmp2[2], tmp2[3]])
#     print(mp2.t2[tmp1[0], tmp1[1], tmp1[2], tmp1[3]],
#           t2[tmp2[0], tmp2[1], tmp2[2], tmp2[3]])
#     #################
    
    
    ################
    # Selected CI
    ###############
    
    print("\nSelected CI")
    sci_solver = hci.SCI(mol)
    sci_solver.select_cutoff = sci_select_cutoff
    sci_solver.ci_coeff_cutoff = sci_ci_coeff_cutoff

    nmo = mf.mo_coeff.shape[1]
    h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
    h2 = ao2mo.full(mol, mf.mo_coeff)
    e_sci, civec = sci_solver.kernel(h1, h2, mf.mo_coeff.shape[1], mol.nelectron, ecore=mol.energy_nuc())
    print("e_sci", e_sci)
    print("Total number of selected CI determinants: {}".format(len(civec[0])))

    # For conversion of SCI to FCI vector for comparison
    neleca = int(nelec/2)
    nelecb = int(nelec/2)

    # Transforming SCI vector to FCI format
    strsa = cistring.gen_strings4orblist(range(norb), neleca)
    stradic = dict(zip(strsa,range(strsa.__len__())))
    strsb = cistring.gen_strings4orblist(range(norb), nelecb)
    strbdic = dict(zip(strsb,range(strsb.__len__())))
    na = len(stradic)
    nb = len(strbdic)
    ndet = len(civec[0])
    fcivec = np.zeros((na,nb))
    for idet, (stra, strb) in enumerate(civec[0]._strs.reshape(ndet,2,-1)):
        ka = stradic[stra[0]]
        kb = strbdic[strb[0]]
        fcivec[ka,kb] = civec[0][idet] # SCI coefficients array in FCI format (with alpha and beta strings) 


    # Obtaining spin orbital occupation numbers for SCI
    det_dict_sci_spin_orb = dict()
    
    for alpha_addr in range(fcivec.shape[0]):
        for beta_addr in range(fcivec.shape[0]):
            # In pyscf the lower energy orbitals are to the RIGHT!!! so we have to reverse the strings
            # converting to strings with occ numbers
            a_str = bin(cistring.addrs2str(norb, int(neleca),[alpha_addr])[0])[2:].rjust(norb, '0')[::-1] 
            b_str = bin(cistring.addrs2str(norb, int(nelecb),[beta_addr])[0])[2:].rjust(norb, '0')[::-1]
            spin_orb_string = a_str + b_str
            det_dict_sci_spin_orb[spin_orb_string] = fcivec[alpha_addr, beta_addr]

    filtered_sci_dets_spin_orb = filter_dets_from_dict(det_dict_sci_spin_orb, det_thresh)        
    print("Number of SCI spin orbitals configurations with CI coefficients > {} is {}".format(det_thresh, len(filtered_sci_dets_spin_orb.keys())))

#     # Filter amps using SCI
    thrown_out_dets = []
    thrown_out_amps = []
    for key in det_dict_sci_spin_orb.keys():
        # loop over all determinants
        if key not in filtered_sci_dets_spin_orb.keys():
            # condition if this determinant was filtered out
            thrown_out_dets.append(key) # this gives us list of dets that were filtered out
            if key in amp_dict.keys():
                thrown_out_amps.append(amp_dict[key]) 
    
    # Creating list of t2 amplitudes to keep after SCI screening
    # matches amps with dets 
    amps_t2_to_keep = []
    n_small_t2_amps_kept = 0
    for key in filtered_sci_dets_spin_orb.keys():
        if key in amp_dict.keys(): # why does this have to be here?
            idx = det_vs_amp_ind[key]
            amps_t2_to_keep.append(idx)
            conv_idx = convert_t2_exc_indices_to_occ_virt_ind(idx, nelec, norb)
#             print("idx = ", idx)
#             print("conv idx", conv_idx)
            if np.abs(t2[conv_idx[0], conv_idx[2], conv_idx[1], conv_idx[3]]) < mp2_thresh:
                n_small_t2_amps_kept +=1
        else:
#             print("KEY {} NOT IN AMP DICT!!!".format(key))
            pass

#     print("t2 amps to keep:", amps_t2_to_keep)

    # Creating list of t1 amplitudes to keep after SCI screening
    # matches amps with dets 
    amps_t1_to_keep = []
    n_small_t1_amps_kept = 0
    t1_filtered = np.zeros(np.shape(t1))
    for key in filtered_sci_dets_spin_orb.keys():
        if key in amp_dict_t1.keys():
            idx = det_vs_amp_t1_ind[key]
            conv_idx = convert_t1_exc_indices_to_occ_virt_ind(idx, nelec, norb)
#             print("idx", idx)
#             print("conv_idx", conv_idx)
            amps_t1_to_keep.append(idx)

            if np.abs(t1[conv_idx[0], conv_idx[1]]) < mp2_thresh:
                n_small_t1_amps_kept +=1 

#     print("t1 amps to keep", amps_t1_to_keep)

    # t1_filtered and t2_filtered indices are not the same format as excitations!
    
    # Creating array for t2 amplitudes where we do prescreening using selected CI
    t2_filtered_sci = np.zeros(np.shape(t2))
    for idx in amps_t2_to_keep:
        conv_idx = convert_t2_exc_indices_to_occ_virt_ind(idx, nelec, norb)
        t2_filtered_sci[conv_idx[0], conv_idx[2], conv_idx[1], conv_idx[3]]\
        = t2[conv_idx[0], conv_idx[2], conv_idx[1], conv_idx[3]]

    # Creating array for t1 amplitudes where we do prescreening using selected CI
    t1_filtered_sci = np.zeros(np.shape(t1))
    for inds in amps_t1_to_keep:
        conv_inds = convert_t1_exc_indices_to_occ_virt_ind(inds, nelec, norb)
        t1_filtered_sci[conv_inds[0], conv_inds[1]] = t1[conv_inds[0], conv_inds[1]]

    # Results to save for analysis

    # CCSD Energies after filtering amplitudes
    e_ccsd_mp2_filter = mycc.energy(t1, t2_filtered_mp2) + e_hf
    e_ccsd_sci_filter = mycc.energy(t1, t2_filtered_sci) + e_hf
#     e_ccsd_sci_filter = mycc.energy(t1_filtered_sci, t2_filtered_sci) + e_hf
    e_ccsd = mycc.energy(t1, t2) + e_hf

    # Errors
    sci_err_wrt_fci = e_sci[0] - e_fci
    ccsd_err_wrt_fci = e_ccsd - e_fci
    err_wrt_ccsd = e_ccsd_sci_filter - e_ccsd
    err_wrt_fci = e_ccsd_sci_filter - e_fci
    err_ccsd_mp2_filter = e_ccsd_mp2_filter - e_fci

    # Amplitudes and determinants
    n_dets_fci = len(det_dict_spin_orb.keys())
    n_dets_sci = len(civec[0])
    n_ccsd_amp = np.shape(t2)[0] * np.shape(t2)[1] * np.shape(t2)[2] * np.shape(t2)[3]
    n_t2_amps_mp2_filter = n_ccsd_amp - len(mp2_amp_inds_smaller_than_thresh)
    n_amps_t2_to_keep = len(amps_t2_to_keep)
    thrown_out_array = np.asarray(thrown_out_amps)
    max_amp_thrown_out = np.max(thrown_out_array)
    n_amps_thrown_out = len(thrown_out_array[thrown_out_array>discarded_amp_thresh])

    print("\nResults:")
    print("{:<40s}{:>30.8e}".format("FCI Energy:", e_fci))
    print("{:<40s}{:>30.8e}".format("CCSD Energy:", e_ccsd))
    
    print("{:<40s}{:>30.8e}".format("SCI Error WRT FCI:", sci_err_wrt_fci))
    print("{:<40s}{:>30.8e}".format("CCSD Error WRT FCI:", ccsd_err_wrt_fci))
    print("\n{:<40s}{:>30.8e}".format("Error WRT CCSD due to cutoff, SCI filter:", err_wrt_ccsd))
    print("{:<40s}{:>30.8e}".format("Error WRT FCI due to cutoff, SCI filter:", err_wrt_fci))

    print("\n{:<40s}{:>30.8e}".format("MP2 cutoff threshold", mp2_thresh))
    print("{:<40s}{:>30.8e}".format("Error of CCSD WRT FCI, MP2 cutoff ", err_ccsd_mp2_filter))
    print("{:<40s}{:>29d}".format("Number of CCSD t2 amplitudes, MP2 filter:", n_t2_amps_mp2_filter))


    print("\n{:<40s}{:>30d}".format("Total number of FCI determinants:", n_dets_fci))
    print("{:<40s}{:>30d}".format("Total number of SCI determinants:", n_dets_sci))
    print("\n{:<40s}{:>30d}".format("Number of CCSD t2 amplitudes", n_ccsd_amp))
    print("{:<40s}{:>30d}".format("Number of t2 amplitudes screened by SCI:", n_amps_t2_to_keep))
    print("{:<30s}{:<2.2e}:{:>21d}".format("Number of kept amplitudes smaller than ", discarded_amp_thresh, n_small_t2_amps_kept))
    print("{:<40s}{:<2.2e} thrown out:{:>10d}".format("Number of amplitudes larger than", discarded_amp_thresh,
        n_amps_thrown_out))
    print("\n{:<40s}{:>30.8e}".format("Largest discarded amplitude:", max_amp_thrown_out))
    
    # DEBUG ##### H2O/STO-3G
#     s1 = [4, 6, 10, 12]
#     s2 = [4, 5, 10, 13]
#     print(all_amps_ccsd[str(s1)], all_amps_ccsd[str(s2)])
#     print(s1, convert_t2_exc_indices_to_occ_virt_ind(s1, nelec, norb))
#     print(s2, convert_t2_exc_indices_to_occ_virt_ind(s2, nelec, norb))
#     s3 = [4, 6, 3, 5]
#     s4 = [4, 5, 3, 6]
#     print(all_amps_ccsd[str(s3)], all_amps_ccsd[str(s4)])
#     print(s3, convert_t2_exc_indices_to_occ_virt_ind(s3, nelec, norb))
#     print(s4, convert_t2_exc_indices_to_occ_virt_ind(s4, nelec, norb))
#     conf1 = '11110011110110'
#     conf2 = '11100111111100'
#     print("fci coeff", det_dict_spin_orb[conf1], det_dict_spin_orb[conf2])
    ##########################
    
#     for key in all_amps_ccsd.keys():
#         if np.abs(amp_dict[key]) > 1e-15:
#             print(key[:norb], key[norb:],":")
#             for tup in all_amps_ccsd[key]:
#                 print(tup)
    
    return amps_t2_to_keep, mp2_amps_to_keep, det_dict_spin_orb, filtered_sci_dets_spin_orb,\
           norb, amp_dict, amp_dict_t1, t2, all_amps_ccsd,\
           t2_inds_ccsd_dict, t1_inds_ccsd_dict, t2_inds_mp2_dict

In [126]:
r = 1.5
basis = 'sto-3g'

# For H2O molecule
theta = 104.5
c,s = np.cos(np.radians(theta)), np.sin(np.radians(theta))
R = np.array(((c, -s), (s, c))) #Rotation matrix
x,y = R @ np.array((0, r))

# atom='O .0 .0 .0; H .0 .0 {}; H .0 {} {}'.format(r, x, y)
# atom = 'Be .0 .0 .0; H .0 .0 {}; H .0 .0 {}'.format(r, -r)
# atom = 'H 0 0 0; H 0 0 {}; H 0 0 {}; H 0 0 {}'.format(r, r*2, r*3)
# atom='Li 0 0 0; H 0 0 {}'.format(r)
# atom='N 0 0 0; N 0 0 {}'.format(r)
atom='H 0 0 0; H 0 0 {}; H 0 0 {}; H 0 0 {}; H 0 0 {}; H 0 0 {}'.format(r, r*2, r*3, r*4, r*5)

# For UCCSD calculations
driver = PySCFDriver(atom=atom, unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis=basis)
molecule = driver.run()


# For pure PySCF calculations
mol = gto.M(atom=atom, basis=basis)
# norb_spatial = int(qubit_op.num_qubits/2)
nelec = molecule.num_alpha + molecule.num_beta

sci_select_cutoff = .0001
sci_ci_coeff_cutoff = .001
det_thresh = 1e-15 # filters SCI selected dets (in FCI format there's a lot of zeros)
discarded_amp_thresh = 1e-06 # for counting amplitudes filtered out by SCI
mp2_thresh = 1e-6 # throw out mp amps smaller than this 

# Running PySCF calculations for filtering amplitudes
t2_sci_filtered, t2_mp2_filtered, det_dict_spin_orb, filtered_sci_dets_spin_orb, norb_spatial, amp_dict_t2,\
    amp_dict_t1, t2_ccsd, all_amps_t2, t2_inds_ccsd_dict, t1_inds_ccsd_dict, t2_inds_mp2_dict\
= run_calcs(r, mol, basis, sci_select_cutoff, sci_ci_coeff_cutoff, det_thresh, discarded_amp_thresh, mp2_thresh)

INFO:qiskit.chemistry.drivers.pyscfd.integrals:PySCF kernel() converged: True, e(hf): -2.75015004418389
INFO:qiskit.chemistry.drivers.pyscfd.integrals:HF Electronic dipole moment: [ -0.         -0.        -42.5188378]
INFO:qiskit.chemistry.drivers.pyscfd.integrals:Nuclear dipole moment: [ 0.         0.        42.5188378]
INFO:qiskit.chemistry.drivers.pyscfd.integrals:Total dipole moment: [0. 0. 0.]


r= 1.5

FCI
FCI energy -2.995565425831942
nelec 6
norb 6
Spin contamination in FCI:
S^2 (4.979684464207637e-29, 1.0)
Number of FCI determinants 400

CCSD
Number of CCSD t1 dets 18
Number of CCSD t2 dets 99

MP2
number of MP2 t2 dets 99

Selected CI
e_sci [-2.99556543]
Total number of selected CI determinants: 200
Number of SCI spin orbitals configurations with CI coefficients > 1e-15 is 200

Results:
FCI Energy:                                            -2.99556543e+00
CCSD Energy:                                           -2.99985149e+00
SCI Error WRT FCI:                                      8.88888962e-12
CCSD Error WRT FCI:                                    -4.28606536e-03

Error WRT CCSD due to cutoff, SCI filter:                0.00000000e+00
Error WRT FCI due to cutoff, SCI filter:               -4.28606536e-03

MP2 cutoff threshold                                    1.00000000e-06
Error of CCSD WRT FCI, MP2 cutoff                      -4.28606536e-03
Number of CCSD t2 amplitu

In [127]:
# Inspecting the 2e integrals array
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
print("shape of 2e- integrals array:", np.shape(h2))
amp_thresh = 1e-6
ci_thresh = 1e-6

# DEBUG FOR H2o/STO-3G
# tmp1 = [4, 6, 2, 5]
# tmp2 = [4, 5, 2, 6]
# print(h2[tmp1[3], tmp1[1], tmp1[0], tmp1[2]])
# print(h2[tmp2[3], tmp2[1], tmp2[0], tmp2[2]])
# print(h2[tmp1[0], tmp1[2], tmp1[1], tmp1[3]])
# print(h2[tmp2[0], tmp2[2], tmp2[1], tmp2[3]])

shape of 2e- integrals array: (12, 12, 12, 12)


# Singles

In [128]:
def convert_str_to_list(inds):
    split_string = inds.strip("]").strip("[").split()
    list_int = [int(el.strip(",")) for el in split_string]
    return list_int
    
    
configs = set()
t1_configs = set()
# Filtering t1 amplitudes
_, t1_amp_vs_det_dict, _ = generate_t1_indices(norb_spatial, nelec)
fci_singles_dict = filter_out_n_order_exc_dets(det_dict_spin_orb, 1)
filtered_fci_singles_dict = filter_dets_from_dict(fci_singles_dict, thresh=ci_thresh)

sci_singles_dict = filter_out_n_order_exc_dets(filtered_sci_dets_spin_orb, 1)

# filtering amplitudes without looking at determinants
filtered_t1 = []
filtered_t1_amps = []
for t1_string in t1_inds_ccsd_dict.keys():
    t1 = convert_str_to_list(t1_string)
#     print(t1)
    if np.abs(h1[t1[0], t1[1]]) > amp_thresh:
        filtered_t1.append(t1)
        filtered_t1_amps.append(h1[t1[0], t1[1]])
print("Number of filtered excitations:", len(filtered_t1))

for key in t1_amp_vs_det_dict.keys():
    configs.add(key)
#     print(key[0:norb_spatial], key[norb_spatial:],
#           t1_amp_vs_det_dict[key],
#           h1[t1_amp_vs_det_dict[key][0], t1_amp_vs_det_dict[key][1]],
#           )
#     if np.abs(h1[t1_amp_vs_det_dict[key][0], t1_amp_vs_det_dict[key][1]]) > amp_thresh:
    # TEST: multiplying integrals by amplitudes
    if np.abs(h1[t1_amp_vs_det_dict[key][0], t1_amp_vs_det_dict[key][1]]) > amp_thresh:
        t1_configs.add(key)
# print("Total number of configurations:", len(configs))
print("\nNumber of singles in FCI:", len(fci_singles_dict.keys()))
print("Number of significant singles in FCI:", len(filtered_fci_singles_dict.keys()))

print("\nSignificant singly excited configurations", len(t1_configs))
print("Number of singles in SCI:", len(sci_singles_dict.keys()))
print("Matching significant configurations with SCI", len(set(sci_singles_dict.keys()).intersection(t1_configs)))
# print(t1_amp_vs_det_dict.keys())
# print("\n", t1_amp_dict)

Number of filtered excitations: 8

Number of singles in FCI: 18
Number of significant singles in FCI: 8

Significant singly excited configurations 8
Number of singles in SCI: 8
Matching significant configurations with SCI 8


The following computations require the integrals stored in the '*chemist*' notation

            h2(i,j,k,l) --> adag_i adag_k a_l a_j

       and the integral values are used for the coefficients of the second-quantized
       Hamiltonian that is built. The integrals input here should be in block spin
       format and also have indexes reordered as follows 'ijkl->ljik'""""

# Doubles

In [129]:
# Filtering amplitudes which are connected by 2-electron integrals elements larger than threshold
_, t2_amp_vs_det_dict, _, _ = generate_t2_indices(norb_spatial, nelec)
# print(t2_inds)
filtered_t2_amp_vs_det_dict = filter_out_n_order_exc_dets(t2_amp_vs_det_dict, 2)

fci_doubles_dict = filter_out_n_order_exc_dets(det_dict_spin_orb, 2)
filtered_fci_doubles_dict = filter_dets_from_dict(fci_doubles_dict, thresh=ci_thresh)

sci_doubles_dict = filter_out_n_order_exc_dets(filtered_sci_dets_spin_orb, 2)
filtered_sci_doubles_dict = filter_dets_from_dict(fci_doubles_dict, thresh=ci_thresh)


# filtering amplitudes without looking at determinants
filtered_t2 = []
filtered_t2_amps = []
for t2_string in t2_inds_ccsd_dict.keys():
    t2 = convert_str_to_list(t2_string)
#     print(t2)
    conv_t2 = convert_t2_exc_indices_to_occ_virt_ind([t2[0], t2[2], t2[1], t2[3]], nelec, norb_spatial)
    if np.abs(h2[t2[3], t2[1], t2[0], t2[2]]) > amp_thresh:
#         print(t2[0], t2[1], t2[2], t2[3])
        if [t2[2], t2[3], t2[0], t2[1]] in filtered_t2:
            print("FLIP")
            pass
        else:
            print(t2, h2[t2[3], t2[1], t2[0], t2[2]], t2_ccsd[conv_t2[0], conv_t2[1], conv_t2[2], conv_t2[3]])
            filtered_t2.append(t2)
            filtered_t2_amps.append(h2[t2[3], t2[1], t2[0], t2[2]])
print("Number of excitations filtered by 2e screening:", len(filtered_t2))




t2_configs = set()

contrib_coeff_doubles_dict_2e = dict()
two_el_doubles = dict()
for key in filtered_t2_amp_vs_det_dict.keys():
    configs.add(key)
    t2_ind = t2_amp_vs_det_dict[key]
#     print("str t2 ind:", str(t2_ind))
#     inds = convert_inds_string_list_ints(str(t2_ind))
#     print(inds)
    if np.abs(h2[t2_ind[3], t2_ind[1], t2_ind[0], t2_ind[2]]) > amp_thresh:
        conv_inds = convert_t2_exc_indices_to_occ_virt_ind([t2_ind[0], t2_ind[2],
                                                            t2_ind[1], t2_ind[3]],
                                                            nelec, norb_spatial)
        contrib_coeff_doubles_dict_2e[key] = t2_ccsd[conv_inds[0], conv_inds[1],
                                                     conv_inds[2], conv_inds[3]]
        two_el_doubles[key] = h2[t2_ind[3], t2_ind[1], t2_ind[0], t2_ind[2]]
        t2_configs.add(key)


print("\nTotal number of doubles in FCI:", len(fci_doubles_dict.keys()))
print("Number of significant doubles in FCI:", len(filtered_fci_doubles_dict.keys()))

fci_intersecting_set = set(filtered_fci_doubles_dict.keys()).intersection(t2_configs)
fci_difference_set = set(filtered_fci_doubles_dict.keys()).symmetric_difference(t2_configs)
print("Matching significant configurations with FCI", len(fci_intersecting_set))
print("Configurations that are in FCI but not in 2e scr") 
for conf in fci_difference_set:
    print(conf[:norb_spatial], conf[norb_spatial:])

print("\nNumber of significant doubles", len(t2_configs))
print("Number of doubles in SCI:", len(sci_doubles_dict.keys()))
print("Number of significant doubles in SCI:", len(filtered_sci_doubles_dict.keys()))
print("Matching significant configurations with SCI", len(set(filtered_sci_doubles_dict.keys()).intersection(t2_configs)))


[0, 3, 6, 9] -0.04285255863241419 -0.0622777738581008
[0, 3, 6, 11] 0.015595953288108543 0.03221743579295692
[0, 4, 6, 10] -0.028672446788292304 -0.055810594546986995
[0, 5, 6, 9] 0.015595953288108536 0.03221743579295692
[0, 5, 6, 11] -0.03957052737383296 -0.12928790778228397
[0, 3, 7, 10] -0.02778892688731682 -0.05413686623508901
[0, 4, 7, 9] -0.018979328613792246 -0.034086701271335174
[0, 4, 7, 11] -0.028193885874553057 -0.10774941962103372
[0, 5, 7, 10] -0.02238449295555301 -0.07568647403141826
[0, 3, 8, 9] 0.006802079342859293 0.024592204374841175
[0, 3, 8, 11] -0.04237342240161005 -0.17194050751576492
[0, 4, 8, 10] -0.018242516340537418 -0.055837564520458696
[0, 5, 8, 9] -0.029030129150755594 -0.10626815258015548
[0, 5, 8, 11] 0.015204145060145727 0.02600126882749334
[1, 3, 6, 10] -0.018979328613792243 -0.034086701271335174
[1, 4, 6, 9] -0.027788926887316823 -0.05413686623508901
[1, 4, 6, 11] -0.022384492955553 -0.07568647403141826
[1, 5, 6, 10] -0.028193885874553064 -0.1077494196

In [183]:
n_triples_to_include = 24
triples_exc = []
thresh_3 = 1e-6
for t2 in filtered_t2:
#     print("\n", t2, ":")
    
    conv_t2 = convert_t2_exc_indices_to_occ_virt_ind([t2[0], t2[2], t2[1], t2[3]], nelec, norb_spatial)
    for t1 in filtered_t1:
        conv_t1 = convert_t1_exc_indices_to_occ_virt_ind(t1, nelec, norb_spatial)
        
        if t1[0] != t2[0] and t1[0] != t2[2] and t1[1] != t2[1] and t1[1] != t2[3]:
#             print(t1, t2, h1[t1[0], t1[1]] * t2_ccsd[conv_t2[0], conv_t2[1], conv_t2[2], conv_t2[3]], conv_t1, conv_t2)
            if np.abs(h1[t1[0], t1[1]] * t2_ccsd[conv_t2[0], conv_t2[1], conv_t2[2], conv_t2[3]]) > thresh_3:
                triples_exc.append(tuple([[t1[0], t1[1], t2[0], t2[1], t2[2], t2[3]],
                                         h1[t1[0], t1[1]] * t2_ccsd[conv_t2[0], conv_t2[1], conv_t2[2], conv_t2[3]]]))
# print(len(triples_exc))
sorted_triples_exc = sorted(triples_exc, key=lambda kv: np.abs(kv[1]), reverse=True)

# print(sorted_triples_exc[:n_triples_to_include])
first_n_triples = [tup[0] for tup in sorted_triples_exc[:n_triples_to_include]]
for el in sorted_triples_exc:
    print(el)

filtered_triples_exc = []
for tup in triples_exc:
    exc = tup[0]
#     print("\norig", exc)
    orb1, orb2, orb3, orb4, orb5, orb6 = exc
    rot_exc = [orb5, orb6, orb1, orb2, orb3, orb4]
    rot_exc_2 = [orb3, orb4, orb5, orb6, orb1, orb2]
    if rot_exc in filtered_triples_exc:
#         print("duplicate!", rot_exc)
        pass
    elif rot_exc_2 in filtered_triples_exc:
#         print("duplicate 2!", rot_exc_2)
        pass
    else:
        filtered_triples_exc.append(exc)
print(len(triples_exc), len(filtered_triples_exc))

([7, 9, 1, 3, 8, 10], 0.030348249682296134)
([1, 3, 2, 4, 7, 9], 0.030348249682296134)
([7, 9, 1, 3, 2, 4], 0.030348249682296134)
([1, 3, 7, 9, 8, 10], 0.030348249682296134)
([7, 9, 0, 3, 8, 11], 0.025257870255739633)
([1, 3, 2, 5, 6, 9], 0.025257870255739633)
([7, 9, 0, 3, 2, 5], 0.025257870255739633)
([1, 3, 6, 9, 8, 11], 0.025257870255739633)
([2, 4, 1, 3, 8, 10], 0.024226815862082376)
([8, 10, 2, 4, 7, 9], 0.024226815862082376)
([8, 10, 1, 3, 2, 4], 0.024226815862082376)
([2, 4, 7, 9, 8, 10], 0.024226815862082376)
([0, 4, 2, 3, 8, 9], 0.02351037274798976)
([6, 10, 2, 3, 8, 9], 0.02351037274798976)
([2, 4, 0, 3, 8, 11], 0.020163198146848586)
([8, 10, 2, 5, 6, 9], 0.020163198146848586)
([8, 10, 0, 3, 2, 5], 0.020163198146848586)
([2, 4, 6, 9, 8, 11], 0.020163198146848586)
([7, 9, 1, 4, 2, 3], 0.019849040506712844)
([1, 3, 7, 10, 8, 9], 0.019849040506712844)
([1, 3, 0, 5, 6, 11], 0.01899225056144229)
([7, 9, 0, 5, 6, 11], 0.01899225056144229)
([8, 10, 1, 4, 2, 3], 0.01584536355240515)

In [153]:
#for H4/STO-3G test triples
# test_triples = [[4, 6, 0, 2, 5, 7], [1, 3, 0, 2, 5, 7], [4, 6, 0, 3, 1, 2], [0, 2, 1, 3, 5, 7], [5, 7, 0, 2, 4, 6]]

In [188]:
quad_exc = []
thresh_4 = 1e-4
for t2 in filtered_t2:
#     print("\n", t2, ":")
    conv_t2 = convert_t2_exc_indices_to_occ_virt_ind([t2[0], t2[2], t2[1], t2[3]], nelec, norb_spatial)
    for t2_2 in filtered_t2:
        if t2[0] != t2_2[0] and t2[0] != t2_2[2] and t2[2] != t2_2[0] and t2[2] != t2_2[2]\
                and t2[1] != t2_2[1] and t2[1] != t2_2[3] and t2[3] != t2_2[1] and t2[3] != t2_2[3]:
#             print(t1, t2, h1[t1[0], t1[1]] * t2_ccsd[conv_t2[0], conv_t2[1], conv_t2[2], conv_t2[3]])
            if np.abs(h2[t2_2[3], t2_2[1], t2_2[0], t2_2[2]]
                      * t2_ccsd[conv_t2[0], conv_t2[1], conv_t2[2], conv_t2[3]]) > thresh_4:
                quad_exc.append([t2[0], t2[1], t2[2], t2[3], t2_2[0], t2_2[1], t2_2[2], t2_2[3]])
# print(quad_exc)

# Running UCCSD

In [156]:
fer_op = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
e_nr = molecule.nuclear_repulsion_energy
core = Hamiltonian(transformation=TransformationType.FULL, qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                   two_qubit_reduction=False, freeze_core=False)
qubit_op, aux_ops = core.run(molecule)
print("UCCSD requires {} qubits".format(qubit_op.num_qubits))

INFO:qiskit.chemistry.core.hamiltonian:Molecule num electrons: [3, 3], remaining for processing: [3, 3]
INFO:qiskit.chemistry.core.hamiltonian:Molecule num spin orbitals: 12, remaining for processing: 12


UCCSD requires 12 qubits


In [ ]:
ee = NumPyMinimumEigensolver(qubit_op)
result = core.process_algorithm_result(ee.run())
exact_e = result['computed_electronic_energy']
print("exact_e", exact_e + e_nr)

In [158]:
init_state = HartreeFock(num_orbitals=core._molecule_info['num_orbitals'],
                    qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction,
                    num_particles=core._molecule_info['num_particles'])

In [191]:
var_form = UCCSD(num_orbitals=core._molecule_info['num_orbitals'],
                   num_particles=core._molecule_info['num_particles'],
                   active_occupied=None, active_unoccupied=None, initial_state=init_state,
                   qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction,
                   num_time_slices=1,
#                    single_exc_op_list=filtered_t1,
#                    double_exc_op_list=filtered_t2,
#                    triple_exc_op_list=first_n_triples,
                )


INFO:qiskit.chemistry.components.variational_forms.uccsd:Singles excitations with alphas and betas orbitals are used.
INFO:qiskit.chemistry.components.variational_forms.uccsd:Singles and Doubles excitations are used.


In [ ]:
# optimizer = L_BFGS_B()
# optimizer = COBYLA()
optimizer = SLSQP()
initial_point = None
# initial_point = np.array([ 1.61257439e-03, -6.51052827e-03, -5.20528843e-03,  2.21419844e-03,
#         1.59693253e-03, -6.63803102e-03, -5.14638915e-03,  2.07597692e-03,
#        -6.43226224e-02,  2.93060938e-02, -5.60833833e-02,  2.96490191e-02,
#        -1.22617449e-01, -5.47737392e-02, -3.51199010e-02, -9.88357019e-02,
#        -6.91770018e-02,  2.44078411e-02, -1.39406208e-01, -5.57114013e-02,
#        -8.67479115e-02,  2.26641076e-02, -3.58132229e-02, -5.40495914e-02,
#        -7.02716903e-02, -9.94501153e-02, -9.73148618e-02, -5.77803842e-02,
#        -1.69439051e-01, -5.69929316e-02, -4.78307336e-02, -1.72802268e-01,
#        -1.17893915e-01, -4.75825531e-02, -3.05107436e-02,  2.42761199e-02,
#        -8.88747966e-02, -5.66815678e-02, -1.40629588e-01,  2.33404583e-02,
#        -1.18633915e-01, -1.73517492e-01, -3.11050308e-02, -4.85471615e-02,
#        -3.48359244e-01,  2.23258558e-02, -8.91393315e-02,  2.24880785e-02,
#        -4.96998653e-02, -1.23214215e-02,  1.23222189e-02, -1.36174695e-02,
#         1.36162289e-02, -3.26424468e-02,  3.26408663e-02, -3.30315424e-02,
#         3.30284987e-02, -1.09377493e-02,  1.09377415e-02, -1.23254898e-02,
#         1.23266028e-02, -1.36068618e-02,  1.36059900e-02, -3.26473958e-02,
#         3.26468708e-02, -3.30316710e-02,  3.30308699e-02, -1.09251876e-02,
#         1.09283660e-02, -1.01716973e-02, -1.01813693e-02, -1.01813256e-02,
#        -1.01697018e-02, -1.07919708e-02, -1.07677576e-02, -6.22177664e-03,
#        -6.23328942e-03,  1.03478332e-02,  1.03543874e-02,  1.03505349e-02,
#         1.03544881e-02, -6.97384008e-03, -6.98987163e-03,  1.11507238e-02,
#         1.11618657e-02,  6.33901443e-03,  6.30291951e-03, -1.69888381e-04,
#        -1.64769037e-04, 0, 0, 0, 0])
vqe = VQE(qubit_op,  var_form=var_form, initial_point=initial_point, optimizer=optimizer) # particle number and S^2 expectation values

backend_type = 'statevector_simulator'
backend = Aer.get_backend(backend_type)
quantum_instance = QuantumInstance(backend=backend, shots=10000)
ret = vqe.run(quantum_instance)


INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:
==================== Setting of VQE ============================
Algorithm: VQE
-- max_evals_grouped: 1
-- circuit_sampler: None
-- expectation: None
-- user_valid_expectation: False
-- include_custom: False
-- expect_op: None
-- operator: -4.418989709669445 * IIIIIIIIIIII
+ 0.1261072084907491 * IIIIIIIIIIIZ
+ 8.081672668878211e-05 * IIIIIIIIIYZY
+ 8.081672668878211e-05 * IIIIIIIIIXZX
- 0.005362480246610761 * IIIIIIIYZZZY
- 0.005362480246610761 * IIIIIIIXZZZX
+ 0.10185354829377757 * IIIIIIIIIIZI
- 0.007456665126184361 * IIIIIIIIYZYI
- 0.007456665126184361 * IIIIIIIIXZXI
+ 0.006719252517314039 * IIIIIIYZZZYI
+ 0.006719252517314039 * IIIIIIXZZZXI
+ 0.06028660469222616 * IIIIIIIIIZII
+ 0.011909994569891499 * IIIIIIIYZYII
+ 0.011909994569891499 * IIIIIIIXZXII
+ 0.0048708750072491355 * IIIIIIIIZIII
- 0.004252621871228968 * IIIIIIYZYIII
- 0.004252621871228968 * IIIIIIXZXIII
- 0.06163974564778204 * IIIIIIIZIIII
- 0.12472714235959424 * III

INFO:qiskit.aqua.quantum_instance:statevector backend only works with shot=1, changing shots from 10000 to 1.
INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.17.0.dev0+4a7caac
Backend: 'statevector_simulator (AerProvider)', with following setting:
{'basis_gates': ['u1', 'u2', 'u3', 'u', 'p', 'r', 'rx', 'ry', 'rz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 'sx', 't', 'tdg', 'swap', 'cx', 'cy', 'cz', 'csx', 'cp', 'cu1', 'cu2', 'cu3', 'rxx', 'ryy', 'rzz', 'rzx', 'ccx', 'cswap', 'mcx', 'mcy', 'mcz', 'mcsx', 'mcp', 'mcu1', 'mcu2', 'mcu3', 'mcrx', 'mcry', 'mcrz', 'mcr', 'mcswap', 'unitary', 'diagonal', 'multiplexer', 'initialize', 'kraus', 'roerror', 'delay', 'pauli', 'save_expval', 'save_density_matrix', 'save_statevector', 'save_probs', 'save_probs_ket', 'save_amplitudes', 'save_amplitudes_sq'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': None, 'optimization_level': None}
RunConfig(max_credits=10, shots=1)
{'timeout': None}
{}
{}
Measurement mitigation: None
IN

INFO:qiskit.transpiler.runningpassmanager:Pass: FixedPoint - 0.01216 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Optimize1qGatesDecomposition - 54.92616 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: CXCancellation - 20.97011 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Depth - 2.16866 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: FixedPoint - 0.01407 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Optimize1qGatesDecomposition - 53.94506 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: CXCancellation - 18.32294 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Depth - 1.99461 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: FixedPoint - 0.01192 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Optimize1qGatesDecomposition - 52.32716 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: CXCancellation - 17.85111 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Depth - 2.51293 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: FixedPo

INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 626.45698 (ms), eval count: 35
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.09179 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 410.11310 (ms), eval count: 36
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06461 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 408.19716 (ms), eval count: 37
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06199 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 623.60716 (ms), eval count: 38
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06509 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 417.33575 (ms), eval count: 39
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06676 (ms)
INFO:qiskit.aqua.algorithms.mi

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 639.43696 (ms), eval count: 78
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 410.29692 (ms), eval count: 79
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06390 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937786] - 409.19995 (ms), eval count: 80
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06700 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 410.13908 (ms), eval count: 81
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-5.81937787] - 405.37000 (ms), eval count: 82
INFO:qiskit.compiler.assembler

...

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06509 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 445.50300 (ms), eval count: 418
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.08297 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 660.44188 (ms), eval count: 419
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06700 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 439.12292 (ms), eval count: 420
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06294 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 433.86793 (ms), eval count: 421
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.07105 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 428.75314 (ms), eval count: 422
INFO:qiskit.compiler.asse

INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 452.03710 (ms), eval count: 460
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06795 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 462.77809 (ms), eval count: 461
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06890 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 457.73196 (ms), eval count: 462
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06104 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 624.66002 (ms), eval count: 463
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.10300 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05603648] - 411.83496 (ms), eval count: 464
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.10228 (ms)
INFO:qiskit.aqua.algorith

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06390 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 456.61211 (ms), eval count: 503
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.09084 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 444.70191 (ms), eval count: 504
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.07415 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 453.73702 (ms), eval count: 505
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.08988 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 435.09912 (ms), eval count: 506
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06914 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 641.07394 (ms), eval count: 507
INFO:qiskit.compiler.asse

INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 429.89087 (ms), eval count: 545
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 437.95514 (ms), eval count: 546
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 614.83431 (ms), eval count: 547
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06175 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 403.61285 (ms), eval count: 548
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 418.26892 (ms), eval count: 549
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorith

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06223 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 401.07775 (ms), eval count: 588
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06604 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 398.66376 (ms), eval count: 589
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06771 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 624.30501 (ms), eval count: 590
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06509 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05742447] - 395.99586 (ms), eval count: 591
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05698 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-6.05881784] - 454.80776 (ms), eval count: 592
INFO:qiskit.compiler.asse

In [ ]:
print('computed e', ret['optimal_value'])
print('computed e + e_nr', ret['optimal_value'] + e_nr)

In [ ]:
print("\nResults:")
print(ret)

LiH SLSQP: -8.940707053286438

BeH2 SLSQP: -15.575474535062831 (with nuclear repulsion)

H4: R = 1.5 Ang
    -1.9961503255188005 exact
    -1.9947826951712004 (with e_nr) filtered S+D
    -1.994792093974596 no filter
    -1.995960728824688 S+D+T
    
    -1.9959594893199122 S+D+T (10 out of 16 largest triples)
    -1.995960720880353  S+D+T (8 out of 16 largest triples)
    -1.995960525173107  S+D+T (6 out of 16 largest triples)
    -1.995632297565844  S+D+T (5 test_triples, one of each degenerate)
    -1.9952862175600417 S+D+T (4 out of 16 largest triples)
    -1.9952866035431018 S+D+T (2 out of 16 largest triples)
    -1.9959605845961215 S+D+T (minus dup triples)

    R = 2.0 Ang   
    -1.8977806459898603 exact
    -1.8933180088285142 S+D filtered 16 params
    -1.8933691975264741 S+D 26 params
    -1.897179920507111 S+D+T

H6/STO-3G R = 1.5 Ang

exact          -2.995565425831958
SD full        -2.9907905720513766
SD filtered    -2.9907122652743867
SDT (first 4)  -2.9911539117729093
SDT (first 8)  -2.9914195295449932
SDT (first 12) -2.9918686455657166
SDT (first 16) -2.9921898913090885
SDT (first 20) -2.9922562119988267
SDT (first 24) -2.9922571510883467

In [163]:
#filtered doubles + 12 triples
opt_values = np.array([ 0.00175695, -0.00628434, -0.00531325,  0.00124393,  0.00174573,
       -0.00641   , -0.0052584 ,  0.00111639, -0.06403625,  0.0289993 ,
       -0.05633387,  0.02934841, -0.12292559, -0.05427299, -0.03497978,
       -0.0987228 , -0.06906491,  0.02398244, -0.13923966, -0.05534101,
       -0.0864445 ,  0.02270834, -0.03564546, -0.05354857, -0.07015643,
       -0.09933613, -0.09684555, -0.05765712, -0.16980674, -0.0568827 ,
       -0.04801851, -0.17316992, -0.11731908, -0.04706502, -0.03056212,
        0.02385142, -0.08855242, -0.05631145, -0.14044262,  0.0234202 ,
       -0.11804873, -0.1738664 , -0.03115661, -0.04802666, -0.34951368,
        0.02197556, -0.08846533,  0.02214037, -0.04903717, -0.0124174 ,
        0.01241775, -0.01366596,  0.01366433, -0.03275275,  0.03275021,
       -0.03314614,  0.033143  , -0.01112627,  0.01112568, -0.01242628,
        0.01242651, -0.01365645,  0.01365556, -0.03275548,  0.03275475,
       -0.03315727,  0.03315612, -0.01111513,  0.01111754, -0.01027287,
       -0.01028293, -0.01028282, -0.01027063, -0.01062468, -0.01060314,
       -0.00610228, -0.00610972,  0.01022409,  0.01023561,  0.01022716,
        0.01023375, -0.00786304, -0.00787345,  0.01225012,  0.01226875, 0, 0, 0, 0])

# Plotting

In [ ]:
n = len(t2_configs)
# n=30
d_sorted_2e_dict = dict(sorted(two_el_doubles.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
d_sorted_fci_dict = dict( sorted(fci_doubles_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
d_sorted_sci_dict = dict( sorted(sci_doubles_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
d_sorted_filtered_fci_doubles_dict = dict( sorted(filtered_fci_doubles_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
d_sorted_filtered_sci_doubles_dict = dict( sorted(filtered_sci_doubles_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))

# d_set_2e_t = set(list(sorted_2e_t_dict.keys())[0:n])
d_set_2e = set(list(d_sorted_2e_dict.keys())[0:n])
d_set_fci = set(list(d_sorted_fci_dict.keys())[0:n])
d_set_sci = set(list(d_sorted_sci_dict.keys())[0:n])

print("Matching configurations with SCI", len(d_set_2e.intersection(d_set_sci)))
print("Matching configurations with FCI", len(d_set_2e.intersection(d_sorted_filtered_fci_doubles_dict)))
doubles_not_in_ccsd = d_set_2e.symmetric_difference(d_set_fci)
for conf in doubles_not_in_ccsd:
    print(conf[:norb_spatial], conf[norb_spatial:])

keys = []
fci_list = []
sci_list = []
two_el = []
t2_amp_list = []
two_el_t = []
n = 0
x = []
print(len(two_el_doubles.keys()))
for key in d_sorted_sci_dict.keys():
#     print(key[:norb_spatial], key[norb_spatial:], #sorted_filtered_fci_triples_dict[key],
#           sci_doubles_dict[key],
#           contrib_coeff_doubles_dict_2e[key])
    keys.append(key[:norb_spatial] + " " + key[norb_spatial:])
    fci_list.append(det_dict_spin_orb[key])
    sci_list.append(sci_doubles_dict[key])
    two_el.append(two_el_doubles[key])
    t2_amp_list.append(contrib_coeff_doubles_dict_2e[key])
    x.append(n)
    n += 1

In [ ]:
%matplotlib notebook
font = {'family' : 'normal',
#         'weight' : 'normal',
        'size'   : 15}

plt.rc('font', **font)
plt.rcParams["figure.figsize"] = (10,7)
plt.scatter(x, np.abs(np.asarray(fci_list))/np.amax(np.abs(np.asarray(fci_list))) , label='FCI coefficients')
plt.scatter(x, np.abs(np.asarray(sci_list))/np.amax(np.abs(np.asarray(sci_list))) , label='SCI coefficients')

plt.scatter(x, np.abs(np.asarray(t2_amp_list))/np.amax(np.abs(np.asarray(t2_amp_list))), label='t2 CCSD amplitudes')
# plt.scatter(x, np.abs(np.asarray(two_el))/np.amax(np.abs(np.asarray(two_el))), label='Hijab')

# plt.plot(x, np.abs(two_el_t)/np.amax(np.abs(two_el_t)), label='2 el t')
plt.xlabel("Configuration")
plt.ylabel("Abs. values of scaled coefficients")
plt.legend()
plt.xticks(x, keys, rotation=90)
plt.tight_layout()

In [ ]:
%matplotlib notebook

plt.rcParams["figure.figsize"] = (10,7)
plt.scatter(x, np.abs(np.asarray(fci_list)) , label='FCI coefficients')
plt.scatter(x, np.abs(np.asarray(sci_list)) , label='SCI coefficients')

# plt.plot(x, np.abs(np.asarray(two_el)), label='2 el')
plt.scatter(x, np.abs(np.asarray(t2_amp_list)), label='t2 CCSD amplitudes')
# plt.plot(x, np.abs(two_el_t)/np.amax(np.abs(two_el_t)), label='2 el t')
plt.legend()
plt.xlabel("Configuration")
plt.ylabel("Abs. values of coefficients")
plt.xticks(x, keys, rotation='vertical')
plt.tight_layout()

# Triples

In [ ]:
# Generating triples from singles
ci_thresh_3 = ci_thresh
# ci_thresh_3 = 1e-5
amp_thresh_3 = amp_thresh
# amp_thresh_3 = 1e-11
fci_triples_dict = filter_out_n_order_exc_dets(det_dict_spin_orb, 3)
filtered_fci_triples_dict = filter_dets_from_dict(fci_triples_dict, thresh=ci_thresh_3)

sci_triples_dict = filter_out_n_order_exc_dets(filtered_sci_dets_spin_orb, 3)
filtered_sci_triples_dict = filter_dets_from_dict(sci_triples_dict, thresh=ci_thresh_3)

t3_configs = set()
contrib_coeff_dict_2e = dict()
contrib_coeff_dict_2e_t = dict()
unfiltered_contrib_coeff_dict_2e = dict()
unfiltered_contrib_coeff_dict_2e_t = dict()

for ini_conf in t1_configs:
    print("Indices for initial conf", ini_conf, t1_amp_vs_det_dict[ini_conf])
    occ = [int(occ_str) for occ_str in ini_conf]
    t3_inds, t3_amp_vs_det_dict, _, _ = generate_t2_indices(norb_spatial, nelec, occ=occ)
    # making sure we only have triple excitations there
    filtered_t3_amp_vs_det_dict = filter_out_n_order_exc_dets(t3_amp_vs_det_dict, 3)
    for key in filtered_t3_amp_vs_det_dict.keys():
        configs.add(key)
        cur_ind = t3_amp_vs_det_dict[key]
#         print(cur_ind)
        h1_h2 = h1[t1_amp_vs_det_dict[ini_conf][0], t1_amp_vs_det_dict[ini_conf][1]]\
              * h2[cur_ind[3], cur_ind[1], cur_ind[0], cur_ind[2]] 
#         h1_h2 = h2[cur_ind[3], cur_ind[1], cur_ind[0], cur_ind[2]] 
    
#         unfiltered_contrib_coeff_dict_2e_t[key] = h1_h2 * amp_dict_t1[ini_conf]
#         # try multiplying this by the amplitude corresponding to starting configuration
# #         if np.abs(h1_h2 * amp_dict_t1[ini_conf]) > amp_thresh:
#         if np.abs(h1_h2 * amp_dict_t1[ini_conf]) > amp_thresh_3:
#             if key in t3_configs:
#                 print("already exists", contrib_coeff_dict_2e_t[key], h1_h2 * amp_dict_t1[ini_conf])
#                 if np.abs(h1_h2 * amp_dict_t1[ini_conf]) - np.abs(contrib_coeff_dict_2e_t[key]) > 1e-04:
#                     print("replacing", contrib_coeff_dict_2e_t[key], "with new", h1_h2 * amp_dict_t1[ini_conf])
#                     contrib_coeff_dict_2e_t[key] = h1_h2 * amp_dict_t1[ini_conf]
#                 else:
# #                     print("Kept old", contrib_coeff_dict_2e[key])
#                     pass       
# #                 print(t1_amp_vs_det_dict[ini_conf][0], t1_amp_vs_det_dict[ini_conf][1], cur_ind)
#             else:
#                 contrib_coeff_dict_2e_t[key] = h1_h2 * amp_dict_t1[ini_conf]
#             t3_configs.add(key)
#         unfiltered_contrib_coeff_dict_2e_t[key] = h1_h2 * amp_dict_t1[ini_conf]

        unfiltered_contrib_coeff_dict_2e_t[key] = h1_h2 
        # try multiplying this by the amplitude corresponding to starting configuration
#         if np.abs(h1_h2 * amp_dict_t1[ini_conf]) > amp_thresh:
        if np.abs(h1_h2) > amp_thresh_3:
            if key in t3_configs:
                print("already exists", contrib_coeff_dict_2e_t[key], h1_h2)
                if np.abs(h1_h2) - np.abs(contrib_coeff_dict_2e_t[key]) > 1e-04:
#                     print("replacing", contrib_coeff_dict_2e_t[key], "with new", h1_h2 )
                    contrib_coeff_dict_2e_t[key] = h1_h2 
                else:
#                     print("Kept old", contrib_coeff_dict_2e[key])
                    pass       
#                 print(t1_amp_vs_det_dict[ini_conf][0], t1_amp_vs_det_dict[ini_conf][1], cur_ind)
            else:
                contrib_coeff_dict_2e_t[key] = h1_h2
            t3_configs.add(key)
        unfiltered_contrib_coeff_dict_2e_t[key] = h1_h2       
        
print("Significant t3 configs", len(t3_configs))
print("Number of triples in FCI:", len(fci_triples_dict.keys()))
print("Number of significant triples in FCI:", len(filtered_fci_triples_dict.keys()))

sci_intersecting_set = set(filtered_sci_triples_dict.keys()).intersection(t3_configs)
sci_difference_set = set(filtered_sci_triples_dict.keys()).symmetric_difference(t3_configs)

fci_intersecting_set = set(filtered_fci_triples_dict.keys()).intersection(t3_configs)
fci_difference_set = set(filtered_fci_triples_dict.keys()).symmetric_difference(t3_configs)
print("Matching significant configurations with FCI:", len(fci_intersecting_set))

print("\nNumber of triples in SCI:", len(sci_triples_dict.keys()))
print("Matching significant configurations with SCI", len(sci_intersecting_set))


In [ ]:
n = len(t3_configs)
# n = 30
sorted_2e_t_dict = dict( sorted(contrib_coeff_dict_2e_t.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
sorted_fci_dict = dict( sorted(fci_triples_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
sorted_sci_dict = dict( sorted(sci_triples_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
sorted_filtered_fci_triples_dict = dict( sorted(filtered_fci_triples_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))
sorted_filtered_sci_triples_dict = dict( sorted(filtered_sci_triples_dict.items(), key=lambda kv: np.abs(kv[1]), reverse=True))

for key in sorted_filtered_sci_triples_dict:
    print(key[:norb_spatial], key[norb_spatial:], t3_amp_vs_det_dict[key])


set_2e_t = set(list(sorted_2e_t_dict.keys())[0:n])
set_fci = set(list(sorted_filtered_fci_triples_dict.keys())[0:n])
set_sci = set(list(sorted_filtered_sci_triples_dict.keys())[0:n])

print(len(set_2e_t.intersection(set_sci)), len(set_2e_t.intersection(set_fci)))
triples_not_generated = set_2e_t.symmetric_difference(set_fci)

print("Missing triples:")
for conf in triples_not_generated:
    print(conf[:norb_spatial], conf[norb_spatial:])
keys = []
fci_list = []
sci_list = []
two_el_t = []
n = 0
x = []
print("\n")
for key in sorted_sci_dict.keys():
#     print(key[:norb_spatial], key[norb_spatial:], #sorted_filtered_fci_triples_dict[key],
#           sorted_sci_dict[key],
#           contrib_coeff_dict_2e[key])
    keys.append(key[:norb_spatial] + " " + key[norb_spatial:])
    fci_list.append(sorted_fci_dict[key])
#     sci_list.append(sorted_sci_dict[key])
    two_el_t.append(contrib_coeff_dict_2e_t[key])
    x.append(n)
    n += 1

In [ ]:
%matplotlib notebook
print(len(x), len(fci_list))
plt.plot(x, np.abs(np.asarray(fci_list))/np.amax(np.abs(np.asarray(fci_list))) , label='fci')
plt.plot(x, np.abs(np.asarray(two_el_t))/np.amax(np.abs(np.asarray(two_el_t))), label='2 el t')
# plt.plot(x, np.abs(np.asarray(sci_list))/np.amax(np.abs(np.asarray(sci_list))) , label='sci')
# plt.plot(x, np.abs(np.asarray(fci_list)) , label='fci')
# plt.plot(x, np.abs(np.asarray(two_el)), label='2 el')
# plt.plot(x, np.abs(np.asarray(sci_list)) , label='sci')
plt.legend()
plt.xticks(x, keys, rotation=65)
plt.tight_layout()

# Quadruples

In [ ]:
# Quadruples
q = 0
t4_configs = set()
ci_thresh_4 = ci_thresh
amp_thresh_4 = amp_thresh
amp_thresh_4 = 5e-5

fci_quad_dict = filter_out_n_order_exc_dets(det_dict_spin_orb, 4)
filtered_fci_quad_dict = filter_dets_from_dict(fci_quad_dict, thresh=ci_thresh_4)

sci_quad_dict = filter_out_n_order_exc_dets(filtered_sci_dets_spin_orb, 4)
filtered_sci_quad_dict = filter_dets_from_dict(sci_quad_dict, thresh=ci_thresh_4)

for ini_conf in t2_configs:
    occ = [int(occ_str) for occ_str in ini_conf]
#     print(occ)
#     print("Indices for initial conf", t2_amp_vs_det_dict[ini_conf])
    t4_inds, t4_amp_vs_det_dict, t4_amp_dict, _ = generate_t2_indices(norb_spatial, nelec, occ=occ)
    filtered_t4_amp_vs_det_dict = filter_out_n_order_exc_dets(t4_amp_vs_det_dict, 4)
    for key in filtered_t4_amp_vs_det_dict.keys():
        configs.add(key)
        cur_ind = t4_amp_vs_det_dict[key]

        h2_h2 = h2[cur_ind[3], cur_ind[1], cur_ind[0], cur_ind[2]]\
              * h2[t2_amp_vs_det_dict[ini_conf][3], 
                   t2_amp_vs_det_dict[ini_conf][1],
                   t2_amp_vs_det_dict[ini_conf][0], 
                   t2_amp_vs_det_dict[ini_conf][2]]
        if np.abs(h2_h2) > amp_thresh_4:
            t4_configs.add(key)

print("Total number of significant quadruple configurations:", len(t4_configs))
            
print("\nNumber of quadruples in FCI:", len(fci_quad_dict.keys()))
print("Number of significant quadruples in FCI:", len(filtered_fci_quad_dict.keys()))

sci_difference_set = set(filtered_sci_quad_dict.keys()).symmetric_difference(t4_configs)
sci_intersecting_set = set(filtered_sci_quad_dict.keys()).intersection(t4_configs)

fci_intersecting_set = set(filtered_fci_quad_dict.keys()).intersection(t4_configs)
fci_difference_set = set(filtered_fci_quad_dict.keys()).symmetric_difference(fci_intersecting_set)
print("Matching significant configurations with FCI", len(fci_intersecting_set))

print("\nNumber of quadruples in SCI:", len(sci_quad_dict.keys()))
print("Number of significant quadruples in SCI:", len(filtered_sci_quad_dict.keys()))
print("Matching significant configurations with SCI", len(sci_intersecting_set))
# print("Configurations that don't match:", difference_set)
 
# for key in difference_set:
#     print(key[0:norb_spatial], key[norb_spatial:], t4_amp_vs_det_dict[key])

In [ ]:
fci_set = set()
fci_set = set.union((set(list(fci_singles_dict.keys()))), fci_set)
fci_set = set.union((set(list(fci_doubles_dict.keys()))), fci_set)
fci_set = set.union((set(list(fci_triples_dict.keys()))), fci_set)
fci_set = set.union((set(list(fci_quad_dict.keys()))), fci_set)          

print("Number of FCI configs up to 4 order", len(fci_set))
print("Number of configs up to 4 order", len(configs))
print("Difference of FCI dict with generated by 2e dict:", len(fci_set.symmetric_difference(configs)))
print("Quadruples FCI intersection with generated by 2e:", len(set(fci_quad_dict.keys()).intersection(t4_configs)))

In [ ]:
print("Number of excitations in SCI of different order (not 1,2,3 or 4):", len(filtered_sci_dets_spin_orb) - len(sci_quad_dict.keys()) - len(sci_singles_dict) -
     len(sci_doubles_dict) - len(sci_triples_dict))
print()


In [ ]:
for n in range(1, 7):
    print("n=", n)
    print(len(filter_out_n_order_exc_dets(filtered_sci_dets_spin_orb, n)))

In [ ]:
filtered_t2 = []
filtered_t2_amps = []
for t2 in t2_inds:
    if np.abs(h2[t2[3], t2[1], t2[0], t2[2]]) > thresh:
        filtered_t2.append(t2)
        filtered_t2_amps.append(h2[t2[3], t2[1], t2[0], t2[2]])
        sorted_filtered_t2 = sorted(zip(filtered_t2, filtered_t2_amps), key=lambda kv: np.abs(kv[1]), reverse=True)
print("Number of t2 amplitudes filtered by 2e integrals amplitudes", len(filtered_t2))
print("Number of t2 amplitudes filtered by SCI", len(t2_sci_filtered))
print("Number of t2 amplitudes filtered by MP2", len(t2_mp2_filtered))


# Printing 2e integrals values for matching indices with amps filtered by SCI
print("\nMatching with SCI")
n_matches_with_SCI_sel = 0
for ind_2e_int_sel in t2_inds: # selected by 2e integrals
    for ind_sci_sel in t2_sci_filtered: # selected by SCI
        if ind_2e_int_sel == ind_sci_sel:
            n_matches_with_SCI_sel += 1
#             print(ind_2e_int_sel, h2[ind_2e_int_sel[3], ind_2e_int_sel[1], ind_2e_int_sel[0], ind_2e_int_sel[2]])
print("Number of matches of 2e filtering with SCI filtering is {}".format(n_matches_with_SCI_sel))


# Printing 2e integrals values for matching indices with amps filtered by MP2
print("\nMatching with MP2")
n_matches_with_mp2_sel = 0
for ind_2e_int_sel in filtered_t2: # selected by 2e integrals
    for ind_mp2_sel in t2_mp2_filtered: # selected by SCI
        if ind_2e_int_sel == ind_mp2_sel:
            n_matches_with_mp2_sel += 1
#             print(ind_2e_int_sel, h2[ind_2e_int_sel[3], ind_2e_int_sel[1], ind_2e_int_sel[0], ind_2e_int_sel[2]])
print("Number of matches of 2e filtering with MP2 filtering is {}".format(n_matches_with_mp2_sel))

In [ ]:
alpha, beta, coeff = create_array_for_heatmap(det_dict_spin_orb)
det_to_num_dict = dict()
alpha_set = set()
beta_set = set()
for x in alpha:
    alpha_set.add(x)
for y in beta:
    beta_set.add(y)

print(len(alpha_set.intersection(beta_set)), len(alpha_set), len(beta_set))

for ind, x in enumerate(alpha_set):
    det_to_num_dict[x] = ind
# print(det_to_num_dict)

# initialize list of lists
data = []
for alpha in alpha_set:
        data.append([det_dict_spin_orb[alpha + beta] for beta in beta_set])
# data = [det_dict_spin_orb[alpha_set + beta] for beta in alpha_set]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = list(alpha_set))
df.index = list(alpha_set)
# print(df)
# sns.heatmap(df)

In [ ]:
# # Generating triples from doubles
# for ini_conf in t2_configs:
# #     print("Indices for initial conf", t1_amp_vs_det_dict[ini_conf])
#     occ = [int(occ_str) for occ_str in ini_conf]

#     t3_inds_2, t3_amp_vs_det_dict_2, t3_amp_dict_2 = generate_t1_indices(norb_spatial, nelec, occ=occ)
#     filtered_t3_amp_vs_det_dict_2 = filter_out_n_order_exc_dets(t3_amp_vs_det_dict_2, 3)
#     for key in filtered_t3_amp_vs_det_dict_2.keys():
# #         print(key, h2_h1 * amp_dict_t2[ini_conf])
#         configs.add(key)
#         cur_ind = t3_amp_vs_det_dict_2[key]

#         h2_h1 = h1[cur_ind[0], cur_ind[1]]\
#               * h2[t2_amp_vs_det_dict[ini_conf][3], t2_amp_vs_det_dict[ini_conf][1],
#                    t2_amp_vs_det_dict[ini_conf][0], t2_amp_vs_det_dict[ini_conf][2]]
#         if np.abs(h2_h1) > amp_thresh_3:
#             if key in t3_configs:
# #                 print(key, "is already in configs with amp", contrib_coeff_dict_2e[key], 
# #                       "new:", h2_h1)
#                 if np.abs(h2_h1 ) > np.abs(contrib_coeff_dict_2e[key]):
#                     contrib_coeff_dict_2e_t[key] = h2_h1 
# #                     print("replaced with new", h2_h1)
#                 else: 
# #                     print("Kept old", contrib_coeff_dict_2e[key])
#                     pass
            
#             else:
#                 t3_configs.add(key)
#                 contrib_coeff_dict_2e[key] = h2_h1 

# print("Significant t3 configs", len(t3_configs))
# print("Number of triples in FCI:", len(fci_triples_dict.keys()))
# print("Number of significant triples in FCI:", len(filtered_fci_triples_dict.keys()))

# sci_intersecting_set = set(filtered_sci_triples_dict.keys()).intersection(t3_configs)
# sci_difference_set = set(filtered_sci_triples_dict.keys()).symmetric_difference(t3_configs)

# fci_intersecting_set = set(filtered_fci_triples_dict.keys()).intersection(t3_configs)
# fci_difference_set = set(filtered_fci_triples_dict.keys()).symmetric_difference(t3_configs)
# print("Matching significant configurations with fCI", len(fci_intersecting_set))


# print("\nNumber of triples in SCI:", len(sci_triples_dict.keys()))
# print("Matching significant configurations with SCI", len(sci_intersecting_set))


In [ ]:
n = 10
print(qubit_op.paulis[n][1])
print(qubit_op.paulis[n][0])
print(qubit_op.paulis[n][1].z)
print(qubit_op.paulis[n][1].x)
print(qubit_op.paulis[n][1].phase)